# Clipping

이전까지 우리는 카메라 앞에 있는 점들만을 다뤘다. 그러나 이제는 카메라를 움직이고 돌릴 수 있게 되었으므로 지금부터는 카메라 뒤에 있는 것들을 식별하는 방법을 알아본다.

## An Overview of the Clipping Process

$$P'_{x} = \frac{P_{x} \cdot d}{P_{z}}$$ 

$$P'_{y} = \frac{P_{y} \cdot d}{P_{z}}$$ 

챕터 9엣 우리는 위와 같은 projecion equation을 사용했다. 이 식에서 우리는 $P_z$가 0이면 0으로 나누는 일어나 공간이 터져버린다고 언급했었다. 또한 음수이면 카메라 뒤에 존재할 것이고 매우 작은 값이어도 오브젝트를 왜곡시킬 수 있다
이러한 문제를 피하기 위해 우리는 projection plane인 $Z = d$ 뒤의 부분을 아예 랜더링하지 않을 것이다. 이러한 $clipping plane$은 우리에게 $clipping volume$의 안과 밖의 점을 분류할 수 있도록 해 준다.

* clipping volume : 실제로 카메라로 보이는 공간

위와 같은 경우에서 clipping volume은 "$Z = d$ 앞에 존재하는 모든 것"이 된다. 우리는 씬에서 clipping volume 안쪽에 있는 것들만을 랜더링 할 것이다.

## The Clipping Volume

![image](https://user-images.githubusercontent.com/74246766/149327003-8fde13bc-9d2f-442c-b131-97ca2114c076.png)

하나의 clipping plane을 사용하여 카메라 뒤의 물체가 랜더링 되지 않도록 하면 올바르게 작동은 되지만 비효율적인 면이 있다. 예를 들어 위 그림처럼 어떤 물체는 카메라 앞에 있지만 viewport 바깥쪽에 위치하기 때문에 보이지 않을 수 있다.
이러한 물체들을 모두 계산하는 것은 낭비이므로 그리지 않는 것이 더 효과적이다.

이를 위해서 추가적인 평면을 정의한다. 이 평면들은 정확하게 viewport 내부의 물체만을 보이도록 한다. 이는 아래 그림과 같은 4개의 평면이다.

![image](https://user-images.githubusercontent.com/74246766/149327921-8f6cea4b-5747-43cb-9026-7ba0d75cf473.png)

각 clipping plane은 공간을 $half-spaces$라 불리우는 두개의 공간으로 나눈다. "inside" half-space는 평면 앞쪽의 공간이고 "out-side" half-space는 평면의 뒷쪽 공간이다. 우리가 정의하는 clipping volume의 "안쪽"은 "inside" half-space들이 교차한 부분을 말한다. 이 경우 clipping volume은 마치 무한히 거대한 피라미드와 같은 공간이 될 것이다.
clipping volume 앞쪽의 물체들을 clip하려면 각 평면에 대해서 물체들을 모두 clipping하면 된다.

## Clipping the Scene Against a Plane

![image](https://user-images.githubusercontent.com/74246766/149328821-7a5e9484-dc42-46ee-afb6-094bea3c25dd.png)


실행하는 작업이 적을수록 랜더링은 빨라진다. 우리는 여러 단계를 거치면서 clipping plane에 대하여 씬을 잘라낼 것이다. 각 단계를 지나며 각 물체의 기하학적인 부분들이 clipping volume의 안쪽인지 바깥쪽인지를 판단하고 이에 따라 해당 기하학적 부분을 허용할 것인지 폐기할 것인지를 결정할 것이다. 현재 단계에서 더 이상 분류할 수 없다면 더 디테일하게 보기 위해 다음 단계로 넘어간다.

![image](https://user-images.githubusercontent.com/74246766/149436312-57d3e4ef-d201-4402-8291-27db406409bd.png)


첫 번째 단계에서는 한번에 모든 물체들을 분류한다. 물체가 clipping volume의 완전히 안쪽에 있다면 허용하고, 완전히 바깥에 있다면 폐기한다.

![image](https://user-images.githubusercontent.com/74246766/149436578-bc1b7807-5c26-463b-bc88-9c334925444b.png)


어느 한쪽으로 완전히 넘어가있지 않은 물체가 있다면, 다음 단계로 넘어가서 해당 물체의 면을 이루는 삼각형들을 독립적으로 분류한다. 만약 삼각형이 완전히 안쪽에 있다면 허용하고, 완전히 바깥쪽에 있다면 폐기한다.

![image](https://user-images.githubusercontent.com/74246766/149436758-b6fe0a69-ef4c-4ade-b786-8f3dc860dea7.png)


마지막으로, 어느 한쪽으로 완전히 넘어가있지 않은 삼각형이 있다면, 다음 단계로 넘어가서 삼각형 자체를 나눈다. 원래의 삼각형은 지워지고, clipping volume 내부의 삼각형 구조를 유지하기 위해 한 개 또는 두 개의 삼각형이 추가된다.

이제 clipping 과정에 대한 전반적인 설명을 끝마쳤다. 이제 직접 구현을 위해 계산과 알고리즘 제작을 진행해 보자.